In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
IMG_SIZE = 224
BATCH_SIZE = 32

In [ ]:
#Define data generators for train,validation and test sets
train_datagen=ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator=train_datagen.flow_from_directory('/content/drive/MyDrive/Brain_Tumor_Detection/Brain_Tumor_Detection/train',
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training')
val_generator=train_datagen.flow_from_directory('/content/drive/MyDrive/Brain_Tumor_Detection/Brain_Tumor_Detection/train',
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation')
test_generator=train_datagen.flow_from_directory('/content/drive/MyDrive/Brain_Tumor_Detection/Brain_Tumor_Detection/test',
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation')

Found 2400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
Found 12 images belonging to 1 classes.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#define the model
model=keras.Sequential([
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(1,activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history=model.fit(train_generator,validation_data=val_generator,epochs=5)

Epoch 1/5
75/75 [==============================] - 296s 4s/step - loss: 0.0129 - accuracy: 0.9958 - val_loss: 0.0103 - val_accuracy: 0.9983
Epoch 2/5
75/75 [==============================] - 313s 4s/step - loss: 0.0062 - accuracy: 0.9987 - val_loss: 0.0204 - val_accuracy: 0.9967
Epoch 3/5
75/75 [==============================] - 296s 4s/step - loss: 0.0153 - accuracy: 0.9967 - val_loss: 0.0110 - val_accuracy: 0.9967
Epoch 4/5
75/75 [==============================] - 314s 4s/step - loss: 0.0304 - accuracy: 0.9887 - val_loss: 0.0471 - val_accuracy: 0.9817
Epoch 5/5
75/75 [==============================] - 313s 4s/step - loss: 0.0098 - accuracy: 0.9962 - val_loss: 0.0015 - val_accuracy: 1.0000


In [ ]:
model.save("Model.h5","label.txt")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

#load the saved model
model=load_model('/content/Model.h5')

In [ ]:
#Load and preprocess the test image
test_image_path = ('/content/drive/MyDrive/Brain_Tumor_Detection/Brain_Tumor_Detection/test/pred/pred11.jpg')
img = image.load_img(test_image_path,target_size=(224,224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis = 0)   #Add batch dimension
img_array /= 255.    #Normalize the pixel values
#Make predictions
prediction = model.predict(img_array)
#print the prediction
if prediction < 0.5:
  print("Prediction: NO tumer (Probability:",prediction[0][0],")")
else:
  print("Prediction: Tumer Present (Probability:",prediction[0][0],")")

1/1 [==============================] - 0s 228ms/step
Prediction: Tumer Present (Probability: 0.99702734 )
